In [1]:
from can_test import *

# in Memory Test

- CAN Frame Rahmen:

| SoF | Base ID | RTR | IDE | FDF | DLC | Data Field | CRC | CRC Delimiter | ACK | EoF | IFS |
|-----|---------|-----|-----|-----|-----|------------|-----|---------------|-----|-----|-----|
| 1   | 11      | 1   | 1   | 1   | 4   | 0 - 64     | 15  | 1             | 2   | 7   | 2   |

SoF - Start of Frame
RTR - Remote Transmission Request
IDE - IDentifier Extension
FDF - FD Format indicator
DLC - Data Length Code
CRC - Cyclic Redundancy Check
ACK - Acknowledgement
EoF - End of Frame
IFS - Inter Frame Space


Order Bit Transmission in Classical Base Frame Format for DataFrame:
    <img alt="can_frame" src="_sources/can_data_frame.png" title="CAN DataFrame" width="1200"/>

Order Bit Transmission in Classical Extended Frame Format for DataFrame:
    <img alt="can_frame" src="_sources/can_ext_data_frame.png" title="CAN DataFrame" width="1300"/>

In [2]:
# Anzahl Header, trailer and bit_stuffing in standard frame
sof, base_id, rtr, ide, fdf, dlc, crc, crc_del, ack, eof, ifs = 1, 11, 1, 1, 1, 4, 15, 1, 2, 7, 2
header = sof + base_id + rtr + ide + fdf + dlc
trailer1 = crc
trailer2 = crc_del + ack + eof + ifs
print('Classical Base Frame Format:')
print('id: {} bit'.format(base_id))
print('header: {} bit \ntrailer1: {} bit \ntrailer2: {} bit \nsum: {} bit'.format(header, trailer1, trailer2, header+trailer1+trailer2))
print('header + trailer1 (bit stuffing): {} bit'.format(header+trailer1))

# worst-case bit stuffing standard frame
max_data = 64
bits_for_stuffing = header + trailer1 + max_data
stuff_bits = bits_for_stuffing / 5
print('max. stuff bits: {}'.format(stuff_bits))

Classical Base Frame Format:
id: 11 bit
header: 19 bit 
trailer1: 15 bit 
trailer2: 12 bit 
sum: 46 bit
header + trailer1 (bit stuffing): 34 bit
max. stuff bits: 19.6


In [3]:
# Anzahl Header, trailer and bit_stuffing in extended frame
sof, base_id, srr, ide, id_extension, rtr, fdf, r0, dlc, crc, crc_del, ack, eof, ifs = 1, 11, 1, 1, 18, 1, 1, 1, 4, 15, 1, 2, 7, 2
header = sof + base_id + rtr + ide + fdf + dlc + srr + r0 + id_extension
trailer1 = crc
trailer2 = crc_del + ack + eof + ifs
print('Classical Extended Frame Format:')
print('id: {} bit'.format(base_id+id_extension))
print('header: {} bit \ntrailer1: {} bit \ntrailer2: {} bit \nsum: {} bit'.format(header, trailer1, trailer2, header+trailer1+trailer2))
print('header + trailer1 (bit stuffing): {} bit'.format(header+trailer1))

# worst-case bit stuffing standard frame
max_data = 64
bits_for_stuffing = header + trailer1 + max_data
stuff_bits = bits_for_stuffing / 5
print('max. stuff bits: {}'.format(stuff_bits))

Classical Extended Frame Format:
id: 29 bit
header: 39 bit 
trailer1: 15 bit 
trailer2: 12 bit 
sum: 66 bit
header + trailer1 (bit stuffing): 54 bit
max. stuff bits: 23.6


In [4]:
# Berechnung möglicher Datenmengen

result_list = list()
for x in range(1, 9):
    result_list.append(calc_data_rate(f_bit=500, n_data_bytes=x, base_or_extended='base'))
    result_list.append(calc_data_rate(f_bit=500, n_data_bytes=x, base_or_extended='extended'))

calc_data_rate(125,2,'base',True)
df_data_rates = pd.DataFrame(result_list)
df_data_rates

base_frame with 2 byte max: 	 n_frame:  72.00 bit 	 t_frame: 576.0000 μs 	 f_data: 32.26 kbit/s
base_frame with 2 byte min: 	 n_frame:  62.00 bit 	 t_frame: 496.0000 μs 	 f_data: 27.78 kbit/s


,frame_format,n_data_bytes,n_frame_max [bit],t_frame_max [ms],f_data_min [kbit/s],n_frame_min [bit],t_frame_min [ms],f_data_max [kbit/s]
0,base,1,62.0,0.124,64.516129,54,0.108,74.074074
1,extended,1,86.0,0.172,46.511628,74,0.148,54.054054
2,base,2,72.0,0.144,111.111111,62,0.124,129.032258
3,extended,2,96.0,0.192,83.333333,82,0.164,97.560976
4,base,3,81.0,0.162,148.148148,70,0.140,171.428571
5,extended,3,105.0,0.210,114.285714,90,0.180,133.333333
6,base,4,91.0,0.182,175.824176,78,0.156,205.128205
7,extended,4,115.0,0.230,139.130435,98,0.196,163.265306
8,base,5,100.0,0.200,200.000000,86,0.172,232.558140
9,extended,5,124.0,0.248,161.290323,106,0.212,188.679245


In [5]:
# Wartezeiten im CAN -> Untersuchung auf "weiche Echtzeit"
# maximale Wartezeit für 1 Frame ergibt sich aus obiger Tabelle t_frame_max = 0,306 ms

calc_realtime()

,can_id,n_bytes,t,t_frame,t_wait,buslast
0,1,8,10,0.306,0.606,3.060
1,2,4,1,0.230,0.830,26.060
2,3,1,1,0.172,0.972,43.260
3,4,8,2,0.306,1.306,58.560
4,5,8,2,0.306,2.006,73.860
5,6,2,5,0.192,2.192,77.700
6,7,2,5,0.192,3.792,81.540
7,8,7,5,0.288,4.088,87.300
8,9,7,5,0.288,7.888,93.060
9,10,8,10,0.306,9.906,96.120


In [6]:
# berechnung mit 250 kbit/s + unsaubere Zuordnung der Prioritäten
n_bytes_list = [6, 8, 7, 8, 2, 8, 2, 7, 7, 8, 8, 6, 4, 2, 1, 4]
t_message_list = [50, 2, 10, 20, 1, 100, 50, 5, 5, 10, 10, 20, 20, 50, 50, 100]

calc_realtime(250, n_bytes_list, t_message_list)

,can_id,n_bytes,t,t_frame,t_wait,buslast
0,1,6,50,0.536,1.136,1.072
1,2,8,2,0.612,1.712,31.672
2,3,7,10,0.576,2.376,37.432
3,4,8,20,0.612,3.512,40.492
4,5,2,1,0.384,3.984,78.892
5,6,8,100,0.612,8.512,79.504
6,7,2,50,0.384,10.184,80.272
7,8,7,5,0.576,14.176,91.792
8,9,7,5,0.576,20.576,103.312
9,10,8,10,0.612,NaN,109.432


Reale Messung:

| sending                                  | receiving                             |
|------------------------------------------|---------------------------------------|
| <img src="_sources/send_250.PNG"/> | <img src="_sources/receive_250.PNG"/> |


bus load:
<img src="_sources/buslast_250.PNG"/>

In [7]:
# berechnung mit 250 kbit/s + ohne kurze zeiten
n_bytes_list = [6, 7, 8, 8, 2, 7, 7, 8, 8, 6, 4, 2, 1, 4]
t_message_list = [50, 10, 20, 100, 50, 5, 5, 10, 10, 20, 20, 50, 50, 100]

calc_realtime(250, n_bytes_list, t_message_list)

,can_id,n_bytes,t,t_frame,t_wait,buslast
0,1,6,50,0.536,1.136,1.072
1,2,7,10,0.576,1.676,6.832
2,3,8,20,0.612,2.312,9.892
3,4,8,100,0.612,2.912,10.504
4,5,2,50,0.384,3.284,11.272
5,6,7,5,0.576,3.876,22.792
6,7,7,5,0.576,4.476,34.312
7,8,8,10,0.612,5.112,40.432
8,9,8,10,0.612,6.812,46.552
9,10,6,20,0.536,7.336,49.232


Reale Messung:

| sending                                  | receiving                                            |
|------------------------------------------|------------------------------------------------------|
| <img src="_sources/send_250_small.PNG"/> | <img src="_sources/receive_250_small.PNG"/> |


bus load:
<img src="_sources/buslast_250_small.PNG"/>

# Fazit

es ergeben sich folgende Kernaussagen:
- max. Nutzbare Datenmenge classical [extended] CAN: 290 kb/s [246 kb/s]
- max. Latenzen classical [extended] CAN: 0.258 ms [0.306 ms]

- sofern die Frame-Einteilung sinnvoll erfolgt, kann ein CAN-Bus mittels zyklischer Daten auch mit einer Buslast von > 90 % weiche Echtzeit garantieren
- Zu Berechnung der Nutzdaten kann daher eine Buslast von 90 % sowie die höchste Nettodatenrate angenommen werden (keine Stuff-Bits)

In [8]:
# Nutzdatenberechnung
datenrate = 500 * 10**3
buslast = 0.9
nutzdaten = 8 * 8
brutto_daten_min = 110
brutto_daten_max = 153

frames_pro_sec_min = datenrate/brutto_daten_max*buslast
frames_pro_sec_max = datenrate/brutto_daten_min*buslast

print('frames pro sec min: {}'. format(frames_pro_sec_min))
print('frames pro sec max: {}'.format(frames_pro_sec_max))

frames pro sec min: 2941.176470588235
frames pro sec max: 4090.9090909090905
